# Importando Pacotes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Importando Dados Consolidados

In [2]:
sheet_id = "175lqFAKF5EPdxYEhIySZzmidtN0cb7I0xikJgNFfZ9g"
btc = pd.read_excel(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx", sheet_name = "Série Histórica")
btc.set_index("Data", inplace = True)
indicadores = pd.read_excel("indicadores.xlsx", index_col = "Data")

# Gerenciamento de Risco

#### Stop Fixo por operação

In [6]:
stop_fixo = pd.DataFrame()
stop_fixo["Retorno"] = btc["Fechamento"].pct_change()
stop_fixo["Alvo_Contínuo"] = pd.Series(stop_fixo["Retorno"])
stop_fixo["Alvo_Binário"] = np.where(stop_fixo["Retorno"] > 0, 1, 0)

STOP = 0.1
stop_fixo["Stop_Long"] = np.where(btc["Abertura"]/btc["Mínima"] > (1+STOP), 1, 0)
stop_fixo["Stop_Short"] = np.where(btc["Máxima"]/btc["Abertura"] > (1+STOP), 1, 0)
stop_fixo[["Alvo_Contínuo", "Alvo_Binário", "Stop_Long", "Stop_Short"]] = stop_fixo[["Alvo_Contínuo", "Alvo_Binário", "Stop_Long", "Stop_Short"]].shift(-1) 
stop_fixo.dropna(inplace = True)

stop_fixo.head()

,Retorno,Alvo_Contínuo,Alvo_Binário,Stop_Long,Stop_Short
Data,,,,,
2010-07-17,NaN,0.733791,1.0,0.0,1.0
2010-07-18,0.733791,-0.058714,0.0,1.0,0.0
2010-07-19,-0.058714,-0.075083,0.0,1.0,0.0
2010-07-20,-0.075083,0.059902,1.0,1.0,0.0
2010-07-21,0.059902,-0.293713,0.0,1.0,0.0


#### Stop Ajustado pela Volatilidade

In [4]:
stop_vol = pd.DataFrame()
stop_vol["Retorno"] = btc["Fechamento"].pct_change()
stop_vol["Alvo_Contínuo"] = pd.Series(stop_vol["Retorno"]).shift(-1)
stop_vol["Alvo_Binário"] = np.where(stop_vol["Retorno"] > 0, 1, 0)

# Distâncias entre a abertura e os extremos do candle
stop_vol["d_max"] = (btc["Máxima"]/btc["Abertura"])-1
stop_vol["d_min"] = (btc["Abertura"]/btc["Mínima"])-1

# Desvio-padrão amostral das distâncias da série 
sd_d_max, sd_d_min = stop_vol["d_max"].std(), stop_vol["d_min"].std()

stop_vol["stop_short"] = np.where(stop_vol["d_max"] > 2*sd_d_max, 1, 0)
stop_vol["stop_short"] = stop_vol["stop_short"].shift()

stop_vol["stop_long"] = np.where(stop_vol["d_min"] > 2*sd_d_min, 1, 0)
stop_vol["stop_long"] = stop_vol["stop_long"].shift()
stop_vol.dropna(inplace = True)

stop_vol.head()

,Retorno,Alvo_Contínuo,Alvo_Binário,d_max,d_min,stop_short,stop_long
Data,,,,,,,
2010-07-18,0.733791,-0.058714,1,0.445043,0.000000,0.0,0.0
2010-07-19,-0.058714,-0.075083,0,0.023872,0.157225,1.0,0.0
2010-07-20,-0.075083,0.059902,0,0.000000,0.101670,0.0,0.0
2010-07-21,0.059902,-0.293713,1,0.066801,0.119234,0.0,0.0
2010-07-22,-0.293713,0.113296,0,0.020073,0.454462,0.0,0.0


# Indicadores Técnicos

In [43]:
inds_tec = pd.DataFrame()
inds_tec["Retorno"] = btc["Fechamento"].pct_change()

# Desvios-Padrão Móveis
inds_tec["STD5"] = inds_tec["Retorno"].rolling(5).std()
inds_tec["STD10"] = inds_tec["Retorno"].rolling(10).std()
inds_tec["STD15"] = inds_tec["Retorno"].rolling(15).std()

# Médias Móveis
inds_tec["MM10"] = btc["Fechamento"].rolling(10).mean()

# Proporções de Corpo dos Candles
inds_tec["PROP"] = (btc["Fechamento"]-btc["Abertura"])/(btc["Máxima"]-btc["Mínima"])

# Direções dos Dias Anteriores
inds_tec["D0"] = np.where(inds_tec["Retorno"] > 0, 1, 0)
inds_tec["D1"] = inds_tec["D0"].shift(1)
inds_tec["D2"] = inds_tec["D0"].shift(2)
inds_tec["D3"] = inds_tec["D0"].shift(3)

# Z-Score Móvel
inds_tec["Z15"] = (btc["Fechamento"]-inds_tec["MM10"])/inds_tec["STD10"]

# RSL
inds_tec["RSL"] = (btc["Fechamento"]/inds_tec["MM10"])-1

# Tendência
tmp = pd.DataFrame()
tmp["MM10"] = inds_tec["MM10"]
tmp["MM100"] = btc["Fechamento"].rolling(100).mean()
tmp["MM200"] = btc["Fechamento"].rolling(200).mean()
tmp.dropna(inplace = True)
tmp["TEND_ALTA"] = np.where((tmp["MM10"] > tmp["MM100"]) & (tmp["MM100"] > tmp["MM200"]), 1, 0)
tmp["TEND_BAIXA"] = np.where((tmp["MM10"] < tmp["MM100"]) & (tmp["MM100"] < tmp["MM200"]), 1, 0)
tmp.drop(["MM10", "MM100", "MM200"], axis = 1, inplace = True)
inds_tec = inds_tec.join(tmp)

df = dados.join(inds_tec)
inds_tec.tail()

,Retorno,STD5,STD10,STD15,MM10,PROP,D0,D1,D2,D3,Z15,RSL,TEND_ALTA,TEND_BAIXA
Data,,,,,,,,,,,,,,
2022-08-22,-0.007898,0.048380,0.033080,0.031567,22829.783155,-0.274018,0,1.0,1.0,0.0,-44086.116255,-0.063881,0.0,1.0
2022-08-23,0.007242,0.049364,0.033420,0.030527,22539.272938,0.187619,1,0.0,1.0,1.0,-30314.400316,-0.044948,0.0,1.0
2022-08-24,-0.005878,0.011449,0.033383,0.030014,22246.651424,-0.218964,0,1.0,0.0,1.0,-25372.652938,-0.038074,0.0,1.0
2022-08-25,0.009553,0.010785,0.034077,0.028490,21992.327099,0.507714,1,0.0,1.0,0.0,-11393.644202,-0.017654,0.0,1.0
2022-08-26,-0.002511,0.007857,0.034171,0.028439,21758.060347,-0.358484,0,1.0,0.0,1.0,-6094.180175,-0.009571,0.0,1.0


# Separação Treinamento e Teste

In [57]:
df.dropna()

,DY_INFLATION,AN_INFLATION,END_ATIVOS,HASH_RATE30,FEE_MEAN,MMULT,MRGO140,MRGO28,MVRV,MVRV_Z,...,MM10,PROP,D0,D1,D2,D3,Z15,RSL,TEND_ALTA,TEND_BAIXA
Data,,,,,,,,,,,,,,,,,,,,,
2011-02-01,0.155102,56.612160,1901,1.498820e-01,3.258084,3.962088,4.259223,4.472763,5.023446,5.562989,...,0.472985,0.578913,1.0,1.0,1.0,0.0,2.314065,0.496305,1.0,0.0
2011-02-02,0.149204,54.459606,1491,1.515172e-01,2.894485,3.934975,3.738760,4.252111,4.955148,5.412498,...,0.500345,0.085702,1.0,1.0,1.0,1.0,2.143098,0.431013,1.0,0.0
2011-02-03,0.150865,55.065768,1273,1.535432e-01,2.164963,3.764052,3.166427,3.801977,4.686634,5.053846,...,0.527994,-0.295465,0.0,1.0,1.0,1.0,1.715342,0.318935,1.0,0.0
2011-02-04,0.182590,66.645333,1505,1.564579e-01,3.008658,4.298699,3.752805,4.627094,5.360298,5.914965,...,0.568094,0.508096,1.0,0.0,1.0,1.0,2.445912,0.427581,1.0,0.0
2011-02-05,0.168189,61.388920,1514,1.591769e-01,4.740692,4.705567,3.924855,5.225930,5.790686,6.488817,...,0.617462,0.649667,1.0,1.0,0.0,1.0,2.979090,0.469483,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-12,0.005024,1.833828,842547,1.073502e+08,161.096062,0.982694,-1.925582,4.605414,2.203437,2.487159,...,43201.776883,-0.539862,0.0,0.0,0.0,1.0,36313.889185,0.028390,0.0,0.0
2021-08-13,0.004725,1.724446,917929,1.069189e+08,167.674949,1.056173,-1.926084,5.981414,2.362590,2.822209,...,44168.902822,0.927556,1.0,0.0,0.0,0.0,101601.796647,0.082933,0.0,0.0
2021-08-14,0.005456,1.991505,724702,1.080999e+08,142.315099,1.038278,-2.201621,5.636149,2.327809,2.745314,...,44898.667011,-0.448117,0.0,1.0,0.0,0.0,59028.728881,0.048961,0.0,0.0


In [63]:
# Período de Treinamento
ini_treino = "2011-02-01"
fim_treino = "2018-12-31"
X_train = df[ini_treino:fim_treino]

# Período de Teste
ini_teste = "2019-01-01"
fim_teste = "2021-07-31"
X_test = df[ini_teste:fim_teste]